In [2]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [7]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\ESTADIC")

In [35]:
df18=pd.read_csv("ESTADIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("ESTADIC_educacao_2021.csv", sep=';')

In [36]:
df18 = df18[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
df21 = df21[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
        

In [37]:
df18['ano']=2018
df21['ano']=2021

In [69]:
df = pd.concat([df18,df21])

In [70]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [72]:
## criando faixa etária
df1= df

In [78]:
### Dataframe do mapa


df1 = df1.rename(columns={'Cod UF':'cod_uf'})
df1 = df1[['ano','cod_uf', 'uf', 'genero', 'faixa_etaria', 'cor_raca', 'escolaridade']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 26
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ano           54 non-null     int64   
 1   cod_uf        54 non-null     int64   
 2   uf            54 non-null     object  
 3   genero        54 non-null     object  
 4   faixa_etaria  54 non-null     category
 5   cor_raca      54 non-null     object  
 6   escolaridade  54 non-null     object  
dtypes: category(1), int64(2), object(4)
memory usage: 3.2+ KB


In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [80]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
    bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
     bigquery.SchemaField('uf','STRING',description='Nome da UF'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('escolaridade','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')

 ]


In [81]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_educacao_mapa')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=862aac1e-fc45-4b31-b69c-31a7956c8a48>